<a href="https://colab.research.google.com/github/itsdrews/Lunar-Lander-v3-AI/blob/main/Lunar_Lander_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Lunar Lander V3 - Deep Q-Learn


```
 pip install swig
 pip install gymnasium[box2d]
```
Imports necessários:


*   OS
*   Random
*   Numpy
*   Torch
*   Gymnasium









In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn # P/ Rede Neural
import torch.optim as optim # Otimizador
import torch.nn.functional as F # Funções de ativação, perda, convolução, etc (ReLU)
import torch.autograd as autograd # Diferenciação automática (gradiente)
from torch.autograd import Variable
from collections import deque, namedtuple

In [ ]:
# Biblioteca Gymnasium
import gymnasium as gym
env = gym.make("LunarLander-v3", continuous=False, gravity=-10.0,
               enable_wind=False, wind_power=15.0, turbulence_power=1.5)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n # 0 - Do Nothing, 1 - Fire left, 2 - Fire down, 3 - Fire right
observation_space = env.observation_space
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)
print('Observation space: ', observation_space)


State shape:  (8,)
State size:  8
Number of actions:  4
Observation space:  Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)


In [ ]:
# Parâmetros de aprendizado
learning_rate = 5e-4
batch_size =100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3


In [ ]:
# Rede Neural

class Network(nn.Module):
  def __init__(self,state_size,number_actions,seed=42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64) # 3 Camadas de neurônios totalmente conectadas
    self.fc3 = nn.Linear(64, number_actions)


  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    q_values = self.fc3(x)
    return q_values